In [43]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
engine = create_engine("postgresql:///kc_housing")
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib as plt

In [80]:
def pullsqldata():
    """This function pulls the necessary columns and rows from the PostGRES DB into a Pandas Dataframe in order 
    to continue with our EDA """
    
    engine = create_engine("postgresql:///kc_housing")
    query = """
                SELECT *
                FROM rpsale AS s
                INNER JOIN resbldg AS b ON CONCAT(s.Major,s.Minor) = CONCAT(b.Major, b.Minor)
                INNER JOIN parcel AS p ON CONCAT(s.Major,s.Minor) = CONCAT(p.Major,p.Minor)
                WHERE EXTRACT(YEAR FROM CAST(documentdate AS DATE)) = 2018
                    AND p.proptype = 'R'
                ;"""
    kc_df = pd.read_sql(sql = query, con = engine)
    return kc_df

In [85]:
df = pullsqldata()

In [82]:
## this converts a saleprice of 0 into a NaN, and then drops all NaN's in saleprice column
df['saleprice'].replace(0, np.nan, inplace= True)
df = df['saleprice'].dropna()

In [32]:
## this replaces entries with a saleprice of NaN with the overall mean of all saleprices
# mean_price = kc_housing_df['saleprice'].mean()
# kc_housing_df['saleprice'] = kc_housing_df['saleprice'].fillna(mean_price)

In [51]:
## LR beginning
Y = kc_housing_df['saleprice']
X = kc_housing_df[['sqft']]

In [50]:
stat_preds_scaled = (X - np.mean(X)) / np.std(X)
sm.OLS(Y, sm.add_constant(stat_preds_scaled)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     5242.
Date:                Tue, 03 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:23:22   Log-Likelihood:            -4.4917e+05
No. Observations:               30273   AIC:                         8.983e+05
Df Residuals:                   30270   BIC:                         8.984e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.241e+05   3863.484    213.302      0.000    8.17e+05    8.32e+05
sqft        3.972e+05   3888.004    102.150      0.000     3.9e+05    4.05e+05
sqftlot    -1.763e+04   3888.004     -4.534      0.000   -2.53e+04      -1e+04
==============================================================================
Omnibus:                    41075.241   Durbin-Watson:                   0.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         22104305.207
Skew:                           7.517   Prob(JB):                         0.00
Kurtosis:                     134.522   Cond. No.                         1.12
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
kc_housing_df.plot(x=kc_housing_df['sqft'],y=kc_housing_df['saleprice'])

KeyError: "None of [Float64Index([1200.0, 4100.0, 1720.0, 2840.0, 2980.0, 3200.0, 2180.0, 2140.0,\n              2340.0, 2350.0,\n              ...\n               810.0, 2470.0, 1730.0,  820.0, 1345.0, 3350.0,  740.0,  980.0,\n              3830.0, 3830.0],\n             dtype='float64', length=30273)] are in the [columns]"